## Prétraiter les données et configurer la caractérisation

Avant de pouvoir exécuter une expérience de Machine Learning automatisé (AutoML), vous devez préparer vos données. Lorsque vous souhaitez entraîner un modèle de classification, il vous suffit de fournir les données d’entraînement.

Une fois que vous avez collecté les données, vous devez créer une ressource de données dans Azure Machine Learning. Pour qu’AutoML comprenne comment lire les données, vous devez créer une ressource de données MLTable qui inclut le schéma des données.

Vous pouvez créer une ressource de données MLTable lorsque vos données sont stockées dans un dossier avec un fichier MLTable. Une fois que vous avez créé la ressource de données, vous pouvez la spécifier en tant qu’entrée avec le code suivant :



In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:input-data-automl:1")

## Exécuter une expérience de Machine Learning automatisé

Pour exécuter une expérience de Machine Learning automatisé (AutoML), vous pouvez configurer et envoyer le travail avec le Kit SDK Python.

Les algorithmes utilisés par AutoML dépendent de la tâche que vous spécifiez. Lorsque vous souhaitez entraîner un modèle de classification, AutoML choisit parmi une liste d’algorithmes de classification :

- Logistic Regression
- Light GBM (Gradient Boosting Machine)
- Arbre de décision
- Forêt aléatoire
- Naive Bayes
- Machine à vecteurs de support linéaire (SVM)
- XGBoost
- Et autres... : https://learn.microsoft.com/fr-fr/azure/machine-learning/how-to-configure-auto-train?view=azureml-api-2#supported-algorithms?azure-portal=true   

### Configurer une expérience AutoML
Lorsque vous utilisez le Kit SDK Python (v2) pour configurer une expérience ou un travail AutoML, vous configurez l’expérience à l’aide de la classe automl. Pour la classification, vous utiliserez la fonction automl.classification comme indiqué dans l’exemple suivant :

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True

### Spécifier la métrique principale
L’un des paramètres les plus importants que vous devez spécifier est primary_metric. La métrique principale est la métrique de performance cible pour laquelle le modèle optimal est déterminé. Azure Machine Learning prend en charge un ensemble de métriques nommées pour chaque type de tâche.

Pour récupérer la liste des métriques disponibles lorsque vous souhaitez entraîner un modèle de classification, vous pouvez utiliser la fonction ClassificationPrimaryMetrics comme illustré ici :

In [ ]:
from azure.ai.ml.automl import ClassificationPrimaryMetrics
 
list(ClassificationPrimaryMetrics)

### Définir les limites
L’entraînement des modèles Machine Learning est coûteux en calcul. Pour réduire les coûts et le temps consacré à l’entraînement, vous pouvez définir des limites à une expérience ou un travail AutoML à l’aide de set_limits().

Il existe plusieurs options pour définir des limites à une expérience AutoML :

timeout_minutes : nombre de minutes après lesquelles l’expérience AutoML complète est terminée.
trial_timeout_minutes : nombre maximal de minutes qu’un essai peut prendre.
max_trials : nombre maximal d’essais ou de modèles qui seront entraînés.
enable_early_termination : indique s’il faut mettre fin à l’expérience si le score ne s’améliore pas à court terme.

In [ ]:
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

### Soumettre un essai AutoML
Vous pouvez soumettre un travail AutoML avec le code suivant :

```Python
# submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
) 
```

Vous pouvez superviser les exécutions de travaux AutoML dans Azure Machine Learning studio. Pour obtenir un lien direct vers le travail AutoML, exécutez le code suivant :

```Python
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
```